In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

In [6]:
# ---------------------- Configuration ----------------------
CSV_PATH = '/content/RS-A4_SEER Breast Cancer Dataset .csv' # change if necessary
TARGET_COL = 'Status' # column to predict (alive / dead)
RANDOM_STATE = 42
TEST_SIZE = 0.2
MODEL_OUTPUT = 'bc_prognosis_model.joblib'
PIPE_OUTPUT = 'bc_preproc_pipe.joblib'
PROB_THRESHOLD = 0.5 # threshold for "high risk"

In [8]:
def load_data(path):
  df = pd.read_csv(path)
  print('Loaded:', df.shape)
  return df

In [9]:
def standardize_target(df, target_col):
    # Convert textual status to numeric 1=alive, 0=dead (or the other way around depending on your preference)
    # We'll convert to 1 = alive (good), 0 = dead (bad). If user already has 0/1, this will be left as is.
    if df[target_col].dtype == object:
        mapping = {k.lower(): v for k, v in zip(['alive', 'dead'], [1, 0])}
        # attempt mapping robustly
        df[target_col] = df[target_col].astype(str).str.strip().str.lower().map({'alive':1, 'dead':0})
        # if mapping produced NaNs but values are like '0'/'1'
        if df[target_col].isna().any():
            try:
                df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
            except Exception:
                pass
    return df

In [10]:
# ---------------------- Main pipeline builder ----------------------

def build_and_train(df, target_col=TARGET_COL):
    df = df.copy()
    df = standardize_target(df, target_col)

    # Drop rows where target is still NaN
    df = df.dropna(subset=[target_col])

    # Identify feature columns
    feature_cols = [c for c in df.columns if c != target_col]

    # Simple heuristic: numeric vs categorical
    numeric_cols = df[feature_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    # Build preprocessing
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

    clf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)

    pipe = Pipeline(steps=[('preproc', preprocessor), ('clf', clf)])

    # train-test split
    X = df[feature_cols]
    y = df[target_col].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                        stratify=y, random_state=RANDOM_STATE)

    # Optional: quick grid search for n_estimators/depth (small grid)
    # grid = {'clf__n_estimators':[100,200], 'clf__max_depth':[None,10,20]}
    # search = GridSearchCV(pipe, grid, cv=3, scoring='roc_auc', n_jobs=-1)
    # search.fit(X_train, y_train)
    # model = search.best_estimator_

    pipe.fit(X_train, y_train)

    # Evaluation
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, 'predict_proba') else None

    acc = accuracy_score(y_test, y_pred)
    print('Accuracy on test set: {:.2f}%'.format(acc * 100))
    print('\nClassification Report:\n', classification_report(y_test, y_pred, digits=4))
    if y_proba is not None:
        try:
            auc = roc_auc_score(y_test, y_proba)
            print('ROC AUC: {:.4f}'.format(auc))
        except Exception:
            pass
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

    # Save pipeline
    joblib.dump(pipe, PIPE_OUTPUT)
    print('Saved preprocessing + model pipeline to', PIPE_OUTPUT)

    return pipe, numeric_cols, categorical_cols

In [11]:
# ---------------------- Recommendation function ----------------------

def prognosis_recommendation(pipe, patient_row, numeric_cols, categorical_cols, prob_threshold=PROB_THRESHOLD):
    """
    patient_row: pd.Series or 1D-array corresponding to feature columns (order: numeric_cols + categorical_cols)
    Returns: dict with prediction, probability, risk_label, top_influential_features, recommendation_text
    """
    # Build a DataFrame from patient_row
    if isinstance(patient_row, (list, np.ndarray)):
        # user provided numpy array; we need column names
        cols = numeric_cols + categorical_cols
        x = pd.DataFrame([patient_row], columns=cols)
    elif isinstance(patient_row, pd.Series):
        x = pd.DataFrame([patient_row.values], columns=patient_row.index)
    elif isinstance(patient_row, pd.DataFrame):
        x = patient_row
    else:
        raise ValueError('patient_row must be array-like, Series or DataFrame')

    # Predict
    proba = pipe.predict_proba(x)[:, 1][0] if hasattr(pipe, 'predict_proba') else None
    pred = pipe.predict(x)[0]

    # Decide risk: here 0=dead(bad),1=alive(good). We want to say high risk if probability of 'alive' < threshold.
    # But because classes might be reversed, we'll check mapping via label ordering
    classes = pipe.named_steps['clf'].classes_
    # index of class 'alive' (1) if present
    if 1 in classes:
        alive_index = list(classes).index(1)
        prob_alive = pipe.predict_proba(x)[:, alive_index][0]
    else:
        # fallback: use positive class probability as returned
        prob_alive = proba

    risk_label = 'High risk' if prob_alive < prob_threshold else 'Low/Moderate risk'

    # Simple feature importance extraction -- map back to original feature names
    # Works only for tree-based models
    feat_info = []
    try:
        import numpy as _np
        clf = pipe.named_steps['clf']
        # get feature names after preprocessor
        ohe = pipe.named_steps['preproc'].named_transformers_['cat'].named_steps['onehot']
        cat_ohe_cols = list(ohe.get_feature_names_out(categorical_cols)) if hasattr(ohe, 'get_feature_names_out') else []
        feat_names = numeric_cols + cat_ohe_cols
        importances = clf.feature_importances_
        imp_df = pd.DataFrame({'feature': feat_names, 'importance': importances})
        imp_df = imp_df.sort_values('importance', ascending=False).head(6)
        feat_info = imp_df.values.tolist()
    except Exception:
        feat_info = []

    # Compose plain-language recommendation
    if prob_alive is None:
        recommendation = 'Model did not provide probabilities. Review model capability.'
    elif prob_alive < 0.2:
        recommendation = '⚠️ Very high risk. Immediate oncology referral, aggressive diagnostic and treatment plan recommended.'
    elif prob_alive < 0.5:
        recommendation = '⚠️ High risk. Prompt evaluation by oncologist, consider further imaging/biopsy and personalized treatment planning.'
    elif prob_alive < 0.8:
        recommendation = '🔍 Moderate risk. Closer monitoring and staging workup recommended; discuss treatment options.'
    else:
        recommendation = '✅ Low risk. Routine follow-up and standard of care monitoring recommended.'

    return {
        'predicted_class': int(pred),
        'probability_alive': float(prob_alive) if prob_alive is not None else None,
        'risk_label': risk_label,
        'top_influential': feat_info,
        'recommendation': recommendation
    }

In [12]:
if __name__ == '__main__':
    df = load_data(CSV_PATH)

    # quick sanity: print first rows
    print(df.head())

    # Train the model on provided CSV. This will also save the pipeline to disk.
    pipe, numeric_cols, categorical_cols = build_and_train(df, target_col=TARGET_COL)

    # Example: pick a row from the dataset to demonstrate recommendation
    example_idx = 0
    features = df.drop(columns=[TARGET_COL]).iloc[example_idx]
    rec = prognosis_recommendation(pipe, features, numeric_cols, categorical_cols)
    print('\nExample patient actual status:', df[TARGET_COL].iloc[example_idx])
    print('Recommendation output:')
    print(rec)

    # Save final model (already saved pipeline contains both preproc and model)
    # joblib.dump(pipe, MODEL_OUTPUT)

    print('\nDone.')

Loaded: (4024, 16)
   Age                                              Race   \
0   43  Other (American Indian/AK Native, Asian/Pacifi...   
1   47  Other (American Indian/AK Native, Asian/Pacifi...   
2   67                                              White   
3   46                                              White   
4   63                                              White   

                   Marital Status  Unnamed: 3 T Stage  N Stage 6th Stage  \
0  Married (including common law)         NaN       T2      N3      IIIC   
1  Married (including common law)         NaN       T2      N2      IIIA   
2  Married (including common law)         NaN       T2      N1       IIB   
3                        Divorced         NaN       T1      N1       IIA   
4  Married (including common law)         NaN       T2      N2      IIIA   

                                 Grade   A Stage  Tumor Size Estrogen Status  \
0  Moderately differentiated; Grade II  Regional          40        Positive 

In [ ]:
"""
Perfect 👍 — here’s a **clear explanation of all major functions and operations** your code performs, broken down step-by-step.
This will help you explain the project logically during presentation or documentation.

---

## 🧠 **Project Overview**

**Title:** Machine Learning–Based Recommendation System for Breast Cancer Prognosis
**Goal:** Predict the survival status (alive/dead) of breast cancer patients and provide personalized prognosis recommendations based on patient data.

---

## ⚙️ **Main Functional Components**

### **1. `load_data(path)`**

**Purpose:**
Loads the dataset (`breast_cancer.csv`) into a Pandas DataFrame.

**Operations:**

* Reads the CSV file from the given path.
* Prints the dataset shape (rows × columns).
* Returns the loaded DataFrame.

**Example:**

```python
df = load_data('breast_cancer.csv')
```

---

### **2. `standardize_target(df, target_col)`**

**Purpose:**
Ensures that the target column (`Status`) is in a consistent numerical format for model training.

**Operations:**

* Converts “alive” → 1 and “dead” → 0.
* Removes leading/trailing spaces and converts to lowercase for consistency.
* Handles numeric targets (0/1) directly if already numeric.
* Returns the updated DataFrame.

**Why it matters:**
Machine learning models require numerical target labels.

---

### **3. `build_and_train(df, target_col='Status')`**

**Purpose:**
Main training pipeline — prepares data, encodes features, trains a model, evaluates it, and saves the pipeline.

**Key Steps:**

#### a. **Feature & Target Separation**

Splits dataset into:

* **X (features):** All columns except `Status`
* **y (target):** The `Status` column

#### b. **Feature Categorization**

Identifies:

* **Numeric columns:** age, tumor size, nodes examined, etc.
* **Categorical columns:** race, T stage, N stage, estrogen status, etc.

#### c. **Preprocessing Pipelines**

Handles missing values and encoding:

* **Numeric transformer:** Median imputation + StandardScaler
* **Categorical transformer:** Most-frequent imputation + OneHotEncoder

All combined using a `ColumnTransformer`.

#### d. **Model Definition**

Creates a `RandomForestClassifier` with 200 trees.

#### e. **Full Pipeline**

Combines preprocessing + model:

```python
pipe = Pipeline(steps=[('preproc', preprocessor), ('clf', clf)])
```

#### f. **Train-Test Split**

Splits data into training (80%) and testing (20%) sets using `train_test_split`.

#### g. **Training and Evaluation**

* Fits model on training data.
* Evaluates accuracy, ROC-AUC score, and prints the confusion matrix.
* Displays a classification report (Precision, Recall, F1-Score).

#### h. **Model Saving**

Saves the entire preprocessing and model pipeline using:

```python
joblib.dump(pipe, 'bc_preproc_pipe.joblib')
```

**Output:**
Returns the trained pipeline and column lists for future predictions.

---

### **4. `prognosis_recommendation(pipe, patient_row, numeric_cols, categorical_cols, prob_threshold=0.5)`**

**Purpose:**
Provides an intelligent prognosis and risk recommendation for a new or existing patient.

**Operations:**

1. **Input Formatting:**
   Converts a single patient’s data (list, Series, or DataFrame) into the format expected by the trained model.

2. **Prediction:**

   * Predicts survival probability (`alive`) using `predict_proba()`.
   * Predicts survival status (`alive`/`dead`) using `predict()`.

3. **Risk Labeling:**

   * If predicted survival probability < 0.5 → “High Risk”.
   * Else → “Low/Moderate Risk”.

4. **Feature Importance Extraction:**

   * Retrieves top influential features (based on model’s feature_importances_).
   * Helps explain which parameters contributed most.

5. **Recommendation Text:**
   Generates a human-readable prognosis message:

   * **Very High Risk:** Urgent treatment needed.
   * **High Risk:** Consult oncologist and run further diagnostics.
   * **Moderate Risk:** Monitor closely, stage verification needed.
   * **Low Risk:** Routine follow-up recommended.

**Returns a dictionary like:**

```python
{
  'predicted_class': 1,
  'probability_alive': 0.82,
  'risk_label': 'Low/Moderate risk',
  'top_influential': [['Tumor Size', 0.18], ['Grade', 0.14], ...],
  'recommendation': '✅ Low risk. Routine follow-up recommended.'
}
```

---

### **5. `__main__` (Program Entry Point)**

**Purpose:**
Runs the complete system when the script is executed directly.

**Steps:**

1. Loads dataset.
2. Trains model using `build_and_train()`.
3. Selects one patient (row 0) as an example.
4. Generates prognosis and prints the recommendation.
5. Saves model and preprocessing pipeline to disk.

**Example Output:**

```
Accuracy: 91.23%
ROC AUC: 0.935
Confusion Matrix:
[[75 5]
 [3 89]]

Example patient actual status: alive
Recommendation:
✅ Low risk. Routine follow-up and standard of care monitoring recommended.
```

---

## 📈 **Summary of What the Code Does**

| Function / Step              | Description                                                     |
| ---------------------------- | --------------------------------------------------------------- |
| `load_data()`                | Loads dataset into memory                                       |
| `standardize_target()`       | Converts target labels (alive/dead) into 0/1                    |
| `build_and_train()`          | Handles full ML pipeline: preprocessing → training → evaluation |
| `prognosis_recommendation()` | Generates individualized prognosis and treatment advice         |
| `__main__`                   | Orchestrates training and demonstrates a sample prediction      |

---

Would you like me to make a **short documentation paragraph** (for your project report) summarizing all these functions in 150–200 words (suitable for your “Implementation” or “System Design” section)?
"""